In [2]:
import requests

In [3]:
r = requests.get("http://www.camara.leg.br/cotas/Ano-2019.csv.zip")

In [4]:
file = open(f"Ano-2019.csv.zip", "wb")
file.write(r.content)
file.close()

In [11]:
from zipfile import ZipFile

zip_file = ZipFile(f"Ano-2019.csv.zip", 'r')
zip_file.extract(member=f"Ano-2019.csv", path=f"reembolso-2019")
zip_file.close()

In [13]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [86]:
DTYPE = {
    'txNomeParlamentar': str,
    'ideCadastro': str,
    'nuCarteiraParlamentar': str,
    'nuLegislatura': str,
    'sgUF': str,
    'sgPartido': str,
    'codLegislatura': str,
    'numSubCota': str,
    'txtDescricao': str,
    'numEspecificacaoSubCota': str,
    'txtDescricaoEspecificacao': str,
    'txtFornecedor': str,
    'txtCNPJCPF': str,
    'txtNumero': str,
    'indTipoDocumento': str,
    'datEmissao': str,
    'vlrDocumento': float,
    'vlrGlosa': str,
    'vlrLiquido': float,
    'numMes': str,
    'numAno': str,
    'numParcela': str,
    'txtPassageiro': str,
    'txtTrecho': str,
    'numLote': str,
    'numRessarcimento': str,
    'nuDeputadoId': str,
    'ideDocumento': str,
}

df = pd.read_csv(
    "reembolso-2019/Ano-2019.csv", 
    delimiter=";", 
    dtype=DTYPE, 
    low_memory=False
)

In [82]:
df.head()

,txNomeParlamentar,cpf,ideCadastro,nuCarteiraParlamentar,nuLegislatura,sgUF,sgPartido,codLegislatura,numSubCota,txtDescricao,numEspecificacaoSubCota,txtDescricaoEspecificacao,txtFornecedor,txtCNPJCPF,txtNumero,indTipoDocumento,datEmissao,vlrDocumento,vlrGlosa,vlrLiquido,numMes,numAno,numParcela,txtPassageiro,txtTrecho,numLote,numRessarcimento,vlrRestituicao,nuDeputadoId,ideDocumento,urlDocumento,situation_date
0,LID.GOV-CD,NaN,NaN,NaN,2015,NaN,NaN,55,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,0,NaN,CRISTIANE FERREIRA EPP,03713903000108,693,None,2019-01-16,111.30,0,111.30,1,2019,0,NaN,NaN,1554246,NaN,NaN,2812,6738121,http://camara.leg.br/cota-parlamentar/nota-fis...,2007-04-12
1,LID.GOV-CD,NaN,NaN,NaN,2019,NaN,NaN,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,0,NaN,CRISTIANE FERREIRA EPP,03713903000108,716,None,2019-02-04,117.19,0,117.19,2,2019,0,NaN,NaN,1570078,NaN,NaN,2812,6769606,http://camara.leg.br/cota-parlamentar/nota-fis...,2005-11-03
2,LID.GOV-CD,NaN,NaN,NaN,2019,NaN,NaN,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,0,NaN,CRISTIANE FERREIRA EPP,03713903000108,723,None,2019-02-25,120.15,0,120.15,2,2019,0,NaN,NaN,1570503,NaN,NaN,2812,6771099,http://camara.leg.br/cota-parlamentar/nota-fis...,2005-11-03
3,LID.GOV-CD,NaN,NaN,NaN,2019,NaN,NaN,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,0,NaN,CRISTIANE FERREIRA EPP,03713903000108,753,None,2019-03-28,151.18,0,151.18,3,2019,0,NaN,NaN,1579375,NaN,NaN,2812,6793690,http://camara.leg.br/cota-parlamentar/nota-fis...,2007-10-30
4,LID.GOV-CD,NaN,NaN,NaN,2019,NaN,NaN,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,0,NaN,PANIFICADORA FERREIRA NETO LTDA - EPP,03713903000108,791,None,2019-05-07,119.90,0,119.90,5,2019,0,NaN,NaN,1590980,NaN,NaN,2812,6821674,http://camara.leg.br/cota-parlamentar/nota-fis...,2005-08-27


In [87]:
df["datEmissao"] = pd.to_datetime(
    df.datEmissao, 
    format='%Y-%m-%d'
)

In [88]:
converters = {
    "0": 'nota_fiscal',
    "1": 'recibo',
    "2": 'despesa_exterior',
    "4": None
}

converters

{'0': 'nota_fiscal', '1': 'recibo', '2': 'despesa_exterior', '4': None}

In [89]:
df["txtCNPJCPF"] = df["txtCNPJCPF"].str.replace(r'\D', '', regex=True)

In [90]:
df.indTipoDocumento.replace(converters, inplace=True)

## Empresas (receita federal)

In [91]:
DTYPE = {
    'cnpj': str,
}

df_empresas = pd.read_csv(
    '2019-11-19-companies.csv.xz', 
    compression='xz',
    dtype=DTYPE,  
)
df_empresas.head(1)

,cnpj,partners,secondary_activities,latitude,longitude,opening,main_activity,neighborhood,share_capital,zip_code,main_activity_code,city_code,judicial_nature_code,additional_address_details,removed_from_simples_since,opened_in,opted_for_comples_in,situation_date,special_situation_date,fax_area_code,phone1_area_code,phone2_area_code,address_type,responsible_federative_entity,hq_or_subsidiary_code,address,situation_reason,city,legal_entity,city_abroad_name,trade_name,number,mei,simples,size,administrative_person_category,name,situation,special_situation,phone,type,state,last_updated
0,08771105000156,"[{""id"": 2, ""name"": ""MARLY MORAIS DA GAMA"", ""cn...",NaN,NaN,NaN,NaN,NaN,REALEZA,10000.0,36900000.0,5611201,4787,2062,KM 51,2016-12-31,2007-04-12,2016-01-01,2007-04-12,NaN,33 33331212,33 33331222,33 33312882,AVENIDA,NaN,1,VITORIA MINAS,0,MANHUACU,NaN,NaN,LANCHONETE E RESTAURANTE PEQUENO PRINCIPE,S/N,0,0,1,49,WERNER COMERCIAL DE ALIMENTOS LTDA,2,NaN,NaN,NaN,MG,NaN


Colunas interessantes:

situation (situação cadastral)
- 1 – NULA: o CNPJ torna-se inválido, seja por inconformidade de dados, fraudes ou outros motivos. 
- 2 – ATIVA: significa que a empresa está em dia com todas as suas pendências, não há nenhuma irregularidade e os pagamentos estão em dia;
- 3 – SUSPENSA: a empresa tem pendências com a Receita Federal, não cumpriu com suas obrigações ou pagamentos ou está sendo investigada por possível fraude;
- 4 – INAPTA: a empresa está a dois anos ou mais com suspensão e não apresentou os devidos dados contábeis;
- 08 – BAIXADA: a empresa foi extinta, seja por solicitação dos sócios ou por ação da Receita;

In [92]:
df_empresas['situation_date'] = pd.to_datetime(df_empresas['situation_date'], format="%Y-%m-%d")

In [93]:
# left.merge(right, on='key')

df_final = df.merge(
    df_empresas,
    how='left',
    left_on='txtCNPJCPF',
    right_on='cnpj'
)

In [94]:
df_final[df_final.cnpj == '09426731000178']

,txNomeParlamentar,cpf,ideCadastro,nuCarteiraParlamentar,nuLegislatura,sgUF,sgPartido,codLegislatura,numSubCota,txtDescricao,numEspecificacaoSubCota,txtDescricaoEspecificacao,txtFornecedor,txtCNPJCPF,txtNumero,indTipoDocumento,datEmissao,vlrDocumento,vlrGlosa,vlrLiquido,numMes,numAno,numParcela,txtPassageiro,txtTrecho,numLote,numRessarcimento,vlrRestituicao,nuDeputadoId,ideDocumento,urlDocumento,cnpj,partners,secondary_activities,latitude,longitude,opening,main_activity,neighborhood,share_capital,zip_code,main_activity_code,city_code,judicial_nature_code,additional_address_details,removed_from_simples_since,opened_in,opted_for_comples_in,situation_date,special_situation_date,fax_area_code,phone1_area_code,phone2_area_code,address_type,responsible_federative_entity,hq_or_subsidiary_code,address,situation_reason,city,legal_entity,city_abroad_name,trade_name,number,mei,simples,size,administrative_person_category,name,situation,special_situation,phone,type,state,last_updated
293529,Pompeo de Mattos,2.834689e+10,73486,516,2019,RS,PDT,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,0,NaN,Point 235,09426731000178,000169,nota_fiscal,2019-08-18,15.0,0,15.0,8,2019,0,NaN,NaN,1627523,NaN,NaN,1458,6905386,https://www.camara.leg.br/cota-parlamentar/doc...,09426731000178,NaN,NaN,NaN,NaN,NaN,NaN,SAO JOSE,5000.0,95680000.0,4729699,8585,2135,LOJA 01,2018-12-31,2008-03-10,2018-01-01,2019-03-11,NaN,NaN,54 32821707,NaN,ESTRADA,NaN,1,RS 235,1,CANELA,NaN,NaN,NaN,437,0,0,1,50,JOSE TERTULIANO MACIEL,8,NaN,NaN,NaN,RS,NaN
293530,Pompeo de Mattos,2.834689e+10,73486,516,2019,RS,PDT,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,0,NaN,Point 235,09426731000178,000169,nota_fiscal,2019-08-18,15.0,0,0.0,8,2019,0,NaN,NaN,1627523,NaN,15.0,1458,6905386,https://www.camara.leg.br/cota-parlamentar/doc...,09426731000178,NaN,NaN,NaN,NaN,NaN,NaN,SAO JOSE,5000.0,95680000.0,4729699,8585,2135,LOJA 01,2018-12-31,2008-03-10,2018-01-01,2019-03-11,NaN,NaN,54 32821707,NaN,ESTRADA,NaN,1,RS 235,1,CANELA,NaN,NaN,NaN,437,0,0,1,50,JOSE TERTULIANO MACIEL,8,NaN,NaN,NaN,RS,NaN


In [97]:
df_final[df_final.situation == 8].query('datEmissao > situation_date').situation_reason.unique()

array([ 1, 67,  2, 71,  6])

In [52]:
df_empresas[df_empresas.situation == 8].head(1)

,cnpj,partners,secondary_activities,latitude,longitude,opening,main_activity,neighborhood,share_capital,zip_code,main_activity_code,city_code,judicial_nature_code,additional_address_details,removed_from_simples_since,opened_in,opted_for_comples_in,situation_date,special_situation_date,fax_area_code,phone1_area_code,phone2_area_code,address_type,responsible_federative_entity,hq_or_subsidiary_code,address,situation_reason,city,legal_entity,city_abroad_name,trade_name,number,mei,simples,size,administrative_person_category,name,situation,special_situation,phone,type,state,last_updated
26,90040981000274,"[{""id"": 2, ""name"": ""BELONI SALETE PERINI RECH""...",NaN,NaN,NaN,NaN,NaN,CENTRO,0.0,99560000.0,4731800,8903,2062,CASA,NaN,2006-11-21,NaN,2013-11-01,NaN,NaN,54 33611150,54 33611612,AVENIDA,NaN,2,SETE DE SETEMBRO,1,SARANDI,NaN,NaN,POSTO TIO ITO,779,0,0,5,49,COMERCIAL DE COMBUSTIVEIS TJ LTDA,8,NaN,NaN,NaN,RS,NaN


situation:

- 1 – NULA
- 2 – ATIVA
- 3 – SUSPENSA
- 4 – INAPTA
- 08 – BAIXADA